# Calculations of the Effect Size (ES) for each microarray study 

###  Using Hedges' g value, an adjusted Cohen's d  value

$$  {Enrichment} = \bar{X_2}-\bar{X_1}$$

Let Group 1 be 6h Sleeping Cerebral Cortex Expression values and Group 2 be 6h SD Cerebral Cortex Expression values 

(S mean - SD mean) **(Logged values, so minus gives ratio)** 

$$  {Pooled\ Standard\  Deviation} = \sqrt\frac{(n_1-1)S_1^2 +(n_2-1)S_2^2}{(n_2 +n_2) -2}  $$  

$$  {Cohen's\ d\ value} = \frac{Enrichment}{Pooled\ Standard\ Deviation} $$

$$  {Correction\ Factor (J\ Factor)} = 1- \frac{3}{4df-1} $$

$$  {Hedges'\ g\ value} = Cohen's\ d\ \text{x}\ J\ $$

$$  {Variance\ in\ d (V_d)} = \frac{n_1- +n_2}{n_1 n_2} + \frac{d^2}{2(n_1 +n_2)}  $$

$$  {Variance\ in\ g (V_g)} = J^2\  \text{x}\ V_d  $$

$$  {Standard\ Error\ in\ g (SE_g)} = \sqrt{V_g}  $$

## Setup working environment and import data

In [1]:
import pandas as pd # Dataframes and file IO
import numpy as np # numerical calculations
%cd /Users/Ella1/Desktop/data sets 430AV2


/Users/Ella1/Desktop/data sets 430AV2


In [2]:
prefix = '430AV2_CerCx_6h_'   # define a prefix to add to column names (making indexing easier later)

In [3]:
# import the data file to a data frame 'df'
df=pd.read_table('DATASET-GSE6514.txt', delimiter='\t',  index_col=0) #,nrows=500)  
df.shape

(45101, 137)

In [4]:
# remove probes that are know to cross-hybridise to more than one target
df =df[~df.index.str.contains('_x_|_s_')]    #   important reverse selector ~ 
df.shape

(40569, 137)

## Look at column names and then setup filters for grouping columns into S and SD groups

In [5]:
df.columns

Index(['Symbol', 'Definition', 'Ensembl_id', 'Entrez_id', 'Unigene_id',
       'GO-Process', 'GO-Function', 'GO-Component', 'Pathway_info',
       'Putative microRNA binding sites',
       ...
       'adjp-HypoT_12hS_vs_HypoT_12hSD', 'GSM149636_HypoT_12hSD.CEL',
       'GSM149637_HypoT_12hSD.CEL', 'GSM149648_HypoT_12hSD.CEL',
       'GSM149649_HypoT_12hSD.CEL', 'GSM149650_HypoT_12hSD.CEL',
       'avg-HypoT_12hSD', 'ANOVA-rawp', 'ANOVA-adjp', 'largest fold'],
      dtype='object', length=137)

In [6]:
# define regular expressions for sleep (S) and sleep dep (SD) filters 
s_filt ='CerCx_6hS.CEL'
sd_filt ='CerCx_6hSD.CEL'

In [7]:
df_s=df.filter(regex= s_filt)
df_s.head()

,GSM149540_CerCx_6hS.CEL,GSM149541_CerCx_6hS.CEL,GSM149542_CerCx_6hS.CEL,GSM149543_CerCx_6hS.CEL,GSM149544_CerCx_6hS.CEL
Probesets,,,,,
1427138_at,6.99143,6.91855,6.90962,7.21366,7.09487
1425600_a_at,7.57241,7.35277,7.98682,7.80865,8.33924
1457168_at,5.66100,5.98464,5.75391,5.56871,5.86512
1450135_at,5.67127,5.32662,5.54216,5.59099,5.72743
1424014_at,8.25093,8.43444,8.45485,8.48692,8.43897


In [8]:
df_sd=df.filter(regex= sd_filt)
df_sd.head()

,GSM149545_CerCx_6hSD.CEL,GSM149546_CerCx_6hSD.CEL,GSM149547_CerCx_6hSD.CEL,GSM149548_CerCx_6hSD.CEL,GSM149549_CerCx_6hSD.CEL
Probesets,,,,,
1427138_at,6.89879,6.98911,6.94709,6.96606,6.88920
1425600_a_at,7.99343,7.84730,6.84302,7.89811,7.37415
1457168_at,5.81278,6.15469,5.47067,5.80854,5.73208
1450135_at,5.68492,5.60002,5.19065,5.47596,5.56918
1424014_at,8.42917,8.09900,8.05030,8.25784,8.26926


## Calculations 

In [9]:
# Enrichment

df[prefix+'Enrich'] = df.filter(regex=sd_filt).mean(axis=1) - df.filter(regex=s_filt).mean(axis=1)

In [10]:
df[prefix+'Enrich'].head()

Probesets
1427138_at     -0.087576
1425600_a_at   -0.220776
1457168_at      0.029076
1450135_at     -0.067548
1424014_at     -0.192108
Name: 430AV2_CerCx_6h_Enrich, dtype: float64

In [11]:
# Calculating Pooled StDev
Scount = df.filter(regex=s_filt).count(axis=1)
SDcount = df.filter(regex=sd_filt).count(axis=1)

StdevS = (Scount-1) * df.filter(regex=s_filt).var(axis=1)
StdevSD = (SDcount-1) * df.filter(regex=sd_filt).var(axis=1)

df[prefix+'poolStDev'] = np.sqrt((StdevS+StdevSD)/(Scount+ SDcount-2))

In [12]:
# Calculating Cohen's d
df[prefix+'Cohens_d'] = df[prefix+'Enrich'] / df[prefix+'poolStDev']

In [13]:
#df[prefix+'poolStDev'].head()
df[prefix+'Cohens_d'] .head()

Probesets
1427138_at     -0.912766
1425600_a_at   -0.508961
1457168_at      0.139661
1450135_at     -0.389462
1424014_at     -1.532826
Name: 430AV2_CerCx_6h_Cohens_d, dtype: float64

In [14]:
# Calculating J value (Correction factor)

df[prefix+'J'] = 1-(3/(4*(Scount+SDcount-1)))                              


In [15]:
# Calculating Hedge's g

df[prefix+'Hedges_g'] = df[prefix+'Cohens_d'] * df[prefix+'J']

In [16]:
#df[prefix+'J'].head()
df[prefix+'Hedges_g'] .head()

Probesets
1427138_at     -0.836702
1425600_a_at   -0.466548
1457168_at      0.128023
1450135_at     -0.357007
1424014_at     -1.405090
Name: 430AV2_CerCx_6h_Hedges_g, dtype: float64

In [17]:
# Calculating Var_d
Scount = df.filter(regex=s_filt).count(axis=1)
SDcount = df.filter(regex=sd_filt).count(axis=1)

Ftop1 = Scount + SDcount
Ftop2 = Scount * SDcount
Fbottom1 = np.square(df[prefix+'Cohens_d']) 
Fbottom2 =  2*(Scount + SDcount)


df[prefix+'Var_d'] = (Ftop1/Ftop2) + (Fbottom1 /Fbottom2)

In [18]:
#check output
df[prefix+'Var_d'].head()

Probesets
1427138_at      0.441657
1425600_a_at    0.412952
1457168_at      0.400975
1450135_at      0.407584
1424014_at      0.517478
Name: 430AV2_CerCx_6h_Var_d, dtype: float64

In [19]:
df[prefix+'Var_g'] = df[prefix+'Var_d'] * np.square(df[prefix+'J'])

In [20]:
# Calculating SEg
df[prefix+'SEg'] = np.sqrt(df[prefix+'Var_g'])

In [21]:
df.sort_values(by= '430AV2_CerCx_6h_Hedges_g', ascending=False, inplace=True)
df

,Symbol,Definition,Ensembl_id,Entrez_id,Unigene_id,GO-Process,GO-Function,GO-Component,Pathway_info,Putative microRNA binding sites,...,ANOVA-adjp,largest fold,430AV2_CerCx_6h_Enrich,430AV2_CerCx_6h_poolStDev,430AV2_CerCx_6h_Cohens_d,430AV2_CerCx_6h_J,430AV2_CerCx_6h_Hedges_g,430AV2_CerCx_6h_Var_d,430AV2_CerCx_6h_Var_g,430AV2_CerCx_6h_SEg
Probesets,,,,,,,,,,,,,,,,,,,,,
1423841_at,Bxdc2,brix domain containing 2,ENSMUSG00000022247,67832,NaN,ribosome biogenesis,protein binding,nucleus,NaN,"mmu-let-7a(miRanda), mmu-let-7b(miRanda), mmu-...",...,1.823457e-14,1.063402,0.616584,0.078437,7.860842,0.916667,7.205772,3.489642,2.932269,1.712387
1417262_at,Ptgs2,Prostaglandin-endoperoxide synthase 2,ENSMUSG00000032487,19225,NaN,keratinocyte differentiation // prostaglandin ...,"oxidoreductase activity, acting on single dono...",endoplasmic reticulum membrane // protein comp...,XPodNet - protein-protein interactions in the ...,"mmu-miR-101(TargetScan|pictar), mmu-miR-101a(m...",...,4.143042e-29,3.221362,1.042468,0.149371,6.979055,0.916667,6.397467,2.835360,2.382490,1.543532
1448651_at,Nudt5,nudix (nucleoside diphosphate linked moiety X)...,ENSMUSG00000025817,53893,NaN,"nucleobase, nucleoside, nucleotide and nucleic...",hydrolase activity // nucleoside-diphosphatase...,NaN,Purine metabolism:KEGG-mmu00230:WP2185(WikiPat...,"mmu-miR-10a(miRanda), mmu-miR-10b(miRanda), mm...",...,3.035651e-04,0.512492,0.411920,0.063142,6.523715,0.916667,5.980072,2.527943,2.124174,1.457455
1435158_at,Rbm12b,RNA binding motif protein 12 B1 [Source:MGI Sy...,ENSMUSG00000046667,72397,NaN,NaN,nucleic acid binding // nucleotide binding // ...,NaN,GenMAPP-mRNA_processing_binding_Reactome // mR...,"mmu-miR-128(TargetScan), mmu-miR-128ab(TargetS...",...,3.115295e-10,0.801740,0.461554,0.073548,6.275574,0.916667,5.752610,2.369142,1.990737,1.410935
1433607_at,Cbln4,cerebellin 4 precursor protein [Source:MGI Sym...,ENSMUSG00000067578,228942,NaN,NaN,NaN,extracellular region // synapse // cell junction,NaN,"mmu-miR-106a(miRanda), mmu-miR-106b(miRanda), ...",...,2.154191e-16,1.676918,1.033240,0.165018,6.261376,0.916667,5.739595,2.360242,1.983259,1.408282
1455599_at,Gfod1,glucose-fructose oxidoreductase domain contain...,ENSMUSG00000051335,328232,NaN,metabolic process // oxidation reduction,electron carrier activity // catalytic activit...,extracellular region,NaN,"mmu-let-7(TargetScan), mmu-miR-128(TargetScan)...",...,3.440578e-23,1.348922,0.422140,0.070494,5.988347,0.916667,5.489318,2.193015,1.842742,1.357476
1434595_at,Trim9,tripartite motif-containing 9 [Source:MGI Symb...,ENSMUSG00000021071,94090,NaN,synaptic vesicle exocytosis,protein binding // metal ion binding // zinc i...,cytoplasm // intracellular // synaptosome,NaN,"mmu-miR-10a(RNAhybrid|miRanda), mmu-miR-10b(RN...",...,7.487603e-19,1.238976,0.554776,0.092804,5.977903,0.916667,5.479745,2.186766,1.837491,1.355541
1418937_at,Dio2,"deiodinase, iodothyronine, type II",ENSMUSG00000007682,13371,NaN,hormone biosynthetic process // thyroid hormon...,thyroxine 5'-deiodinase activity // oxidoreduc...,integral to membrane // membrane,Selenium metabolism/Selenoproteins:WP108(WikiP...,"mmu-miR-1192(miRanda), mmu-miR-122(miRanda), m...",...,6.303485e-31,2.738436,0.906618,0.151735,5.974996,0.916667,5.477080,2.185029,1.836031,1.355002
1427126_at,Hspa1b,heat shock protein 1B,ENSMUSG00000090877,15511,NaN,response to stress // anti-apoptosis // negati...,protein binding // nucleotide binding // ATP b...,mitochondrion // intracellular // mitochondria...,NaN,"mmu-miR-128(miRanda), mmu-miR-130a(miRanda), m...",...,1.119386e-22,3.350738,0.929060,0.159003,5.843032,0.916667,5.356113,2.107051,1.770508,1.330604


In [22]:
df.columns

Index(['Symbol', 'Definition', 'Ensembl_id', 'Entrez_id', 'Unigene_id',
       'GO-Process', 'GO-Function', 'GO-Component', 'Pathway_info',
       'Putative microRNA binding sites',
       ...
       'ANOVA-adjp', 'largest fold', '430AV2_CerCx_6h_Enrich',
       '430AV2_CerCx_6h_poolStDev', '430AV2_CerCx_6h_Cohens_d',
       '430AV2_CerCx_6h_J', '430AV2_CerCx_6h_Hedges_g',
       '430AV2_CerCx_6h_Var_d', '430AV2_CerCx_6h_Var_g',
       '430AV2_CerCx_6h_SEg'],
      dtype='object', length=145)

### Import key file from BioMart and index probesets to MGI gene symbols

In [23]:
dfX=pd.read_table('../FHS project/Sleep notebook Copy/BioMart_Ensmbl_index/mart_export72_430v2430Av2.txt',index_col=[3])
 
dfX.pop('Affy mouse430 2 probeset') # remove 430V2 probeset info (not needed for 430AV2 indexing)
dfX.head(5)

,Ensembl Gene ID,Description,MGI symbol
Affy mouse430a 2 probeset,,,
1417126_a_at,ENSMUSG00000039221,ribosomal protein L22 like 1 [Source:MGI Symbo...,Rpl22l1
NaN,ENSMUSG00000095611,predicted gene 10597 [Source:MGI Symbol;Acc:MG...,Gm10597
1417730_at,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1
1417730_at,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1
NaN,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1


In [24]:
df_Join = df.join(dfX, how='left', sort=True)
df_FINAL1 = df_Join.groupby('MGI symbol').mean()
df_FINAL1[df_FINAL1.index.duplicated()==True]   # checking that no duplicate entries exist in the dataframe

,GSM149516_CerCx_3hS.CEL,GSM149517_CerCx_3hS.CEL,GSM149518_CerCx_3hS.CEL,GSM149519_CerCx_3hS.CEL,GSM149520_CerCx_3hS.CEL,avg-CerCx_3hS,log_fold-CerCx_3hS_vs_CerCx_3hSD,fold-CerCx_3hS_vs_CerCx_3hSD,rawp-CerCx_3hS_vs_CerCx_3hSD,adjp-CerCx_3hS_vs_CerCx_3hSD,...,ANOVA-adjp,largest fold,430AV2_CerCx_6h_Enrich,430AV2_CerCx_6h_poolStDev,430AV2_CerCx_6h_Cohens_d,430AV2_CerCx_6h_J,430AV2_CerCx_6h_Hedges_g,430AV2_CerCx_6h_Var_d,430AV2_CerCx_6h_Var_g,430AV2_CerCx_6h_SEg
MGI symbol,,,,,,,,,,,,,,,,,,,,,


### Columns from the list above can then easily be picked to produce files for use later. Examples below given:
 #### df3 = average S and SD expression for the platform and the log-fold changes
 #### df4 = Hedges g  values and associated variance for Meta-analysis (after indexing)

In [25]:
# df3 = df_FINAL1.loc[:,[u'avg-SD', u'avg-S', u'log_fold-S_vs_SD']]
# df3.columns =[prefix+'avg-SD', prefix+'avg-S', prefix+'log_fold-S_vs_SD']
# df3.to_csv('input_files/430AV2_SymbolExpression_forIndex.csv')

In [26]:
df4 = df_FINAL1.loc[:,[u'430AV2_CerCx_6h_Enrich',u'430AV2_CerCx_6h_Hedges_g', u'430AV2_CerCx_6h_Var_g', u'430AV2_CerCx_6h_SEg']]
df4.to_csv('../FHS project/Sleep notebook Copy/IPython_notebooks/input_files/430AV2_CerCx_6h_SymbolforIndexHedges.csv')

In [27]:
df4.head(10)  # check final ouput

,430AV2_CerCx_6h_Enrich,430AV2_CerCx_6h_Hedges_g,430AV2_CerCx_6h_Var_g,430AV2_CerCx_6h_SEg
MGI symbol,,,,
0610005C13Rik,0.202366,1.095235,0.396088,0.629355
0610008F07Rik,-0.026088,-0.175212,0.337646,0.581073
0610009B22Rik,0.123608,1.099110,0.396513,0.629693
0610009D07Rik,0.112382,0.365181,0.345526,0.587770
0610009O20Rik,-0.006146,-0.035211,0.336173,0.579804
0610010K14Rik,-0.297832,-1.150433,0.402286,0.634260
0610012G03Rik,0.096783,0.151125,0.385851,0.620693
0610031J06Rik,-0.055522,-0.309338,0.340896,0.583863
0610037L13Rik,-0.016128,-0.131396,0.336974,0.580495
